In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("data/application_train.csv")

In [4]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [6]:
df.shape

(307511, 122)

In [9]:
df["TARGET"].value_counts(normalize=True)

TARGET
0    0.919271
1    0.080729
Name: proportion, dtype: float64

In [11]:
# creating the decison column for buisness thiking
df["decision"] = df["TARGET"].map({
     0: 'APPROVE',
    1: 'REJECT'
})

In [15]:

df["decision"].value_counts(normalize=True)

decision
APPROVE    0.919271
REJECT     0.080729
Name: proportion, dtype: float64

In [22]:
# as our data is static we are making it dynamic
#1)creating a time column
#2) split data into branches
#3) treat earliers data as baseline
#4) treat later data as monitoring period

# decision_date colum  added

start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2023-12-31")

df['decision_date'] = np.random.choice(
    pd.date_range(start_date, end_date, freq='D'),
    size=len(df)
)


In [23]:
# sort the date for time analysis
df = df.sort_values("decision_date")

In [32]:
df['month'] = df['decision_date'].dt.to_period('M')


In [ ]:
df["month"]

282183    2020-01
184852    2020-01
260670    2020-01
11149     2020-01
296998    2020-01
           ...   
204828    2023-12
218835    2023-12
302170    2023-12
172983    2023-12
30740     2023-12
Name: month, Length: 307511, dtype: period[M]

In [38]:
approval_trend = (
    df.groupby("month")["decision"].apply(
        lambda x : (x=="APPROVE").mean()
    )
)
approval_trend.head()

month
2020-01    0.923960
2020-02    0.917595
2020-03    0.920357
2020-04    0.924150
2020-05    0.917440
Freq: M, Name: decision, dtype: float64

# splitting into baseline and monitoring

In [ ]:
baseline_months = df["month"].sort_values().unique()[:6]


In [42]:
# spliting the data

baseline_df = df[df["month"].isin(baseline_months)]
monitor_df = df[~df["month"].isin(baseline_months)]

In [43]:
print("Baseline approval rate:",
      (baseline_df['decision'] == 'APPROVE').mean())

print("Monitoring approval rate:",
      (monitor_df['decision'] == 'APPROVE').mean())


Baseline approval rate: 0.9210169137607016
Monitoring approval rate: 0.9190227303964725
